In [234]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction
from heapq import nlargest
from functools import reduce
from operator import add
from scipy.spatial import distance
from numpy import array
from scipy.linalg import svd
import copy
import scipy.spatial.distance as distance
import heapq
import random
import numpy
import time
import string
import json
import re

In [261]:
global recipes_before_removing_element
global flattened_recipes_set
global unique_ingredients
global unique_ingredients_after_removing_hidden

f = open('train.json')
data = json.load(f)

recipe_list = []

for i in data:
    #split with comma to get the cuisine.
    split1=str(i).split(',')
    #clean the strings with split and regex
    split2 = split1[1].split(':')
    cuisine = re.sub(r'[.\W]','',split2[1])
    #get the recipe id
    recipe_split = split1[0].split(': ')
    recipe_id=recipe_split[1]
    #keep only the pepperoni guys
    if cuisine=='italian':
        #splitting with [ because ingredients have the only list in the string
        split3=str(i).split('[')
        #final cut
        ingredients=split3[1].split(',')
        #removing ' from the strings
        ingredients = [s.translate(str.maketrans('','',string.punctuation)) for s in ingredients]
        if len(ingredients)>2:
            recipe_list.append(ingredients)
            
            
#remove whitespaces
for i,obj in enumerate(recipe_list):
    recipe_list[i] = [x.strip() for x in recipe_list[i]]

print(len(recipe_list))            
#make the 2d list 1d so we can count occurences easily
flattened_recipes_set = copy.deepcopy(recipe_list)
flattened_recipes_set = reduce(lambda x,y :x+y ,flattened_recipes_set)


#find the unique ingredients list of all the italian dataset
unique_ingredients = list(numpy.unique(numpy.array(flattened_recipes_set)))
print(len(unique_ingredients))            

#randomize the recipes
random.shuffle(recipe_list)
#get the first 1000 randomized items
set_of_recipes = recipe_list[:1000]



recipes_before_removing_element = copy.deepcopy(set_of_recipes)

hidden_items_to_remove = []

#remove 1 random ingredient from every recipes
for i,obj in enumerate(set_of_recipes):
    #randomise the list and remove the first element
    random.shuffle(set_of_recipes[i])
    hidden_items_to_remove.append(set_of_recipes[i][0])
    set_of_recipes[i].pop(0)


recipes=copy.deepcopy(set_of_recipes)
print(len(hidden_items_to_remove))
#unique ingredients have all the items except the hidden FOR ALL ITALIAN RECIPES
unique_ingredients_without_hidden = list(set(unique_ingredients) - set(hidden_items_to_remove))



7801
2932
1000


In [268]:
vectorized_matrix_m =[]
for i,obj in enumerate(recipe_list):
    vector = [0] * len(unique_ingredients_without_hidden)
    current_recipe = recipe_list[i]
    for j,obj in enumerate(current_recipe):
        if current_recipe[j] in unique_ingredients_without_hidden:
            vector[unique_ingredients_without_hidden.index(current_recipe[j])] = 1
    vectorized_matrix_m.append(vector)

In [269]:
# Popular_without_recipes_ingredients is a dict but without the ingredients 
# of the current recipe of the loop.

score_list = []
#popular is a dictionary with : (ingredient,times igredient used in all the recipes)
popular = dict((x,flattened_recipes_set.count(x)) for x in set(flattened_recipes_set))
for i in recipes:
    popular_without_recipes_ingredients = list(filter(lambda x: x in i, popular))
    max_value = max(popular_without_recipes_ingredients)
    score_list.append(max_value)
most_popular_score_list = score_list

In [270]:
distance_recipes=[]
for elem1 in recipes:
    #list with the jaccard of the recipe elem1 with every other
    jaccard_of_recipe = []
    for elem2 in recipe_list:
        set1, set2 = set(elem1), set(elem2)
        jaccard_of_recipe.append(len(set1 & set2) / len(set1 | set2))
        
    distance_recipes.append(jaccard_of_recipe)

In [ ]:
#getting the length similar indexes for the recipes

length = 64
most_similar = []
scores_ucf = []
most_similar_values = []


for i,obj in enumerate(recipes):
    
    a = numpy.array(distance_recipes[i])
    
    #the Jaccard similarity of an item with itself is always 1.
    #we are going to get the n+1 largest values so we can pop out
    #the one with itself.
    
    #gives the indexes of the n+1 largest values of jaccard
    
    most_similar = heapq.nlargest(length+1, range(len(a)), a.take)
    most_similar.pop(0)
    
    #gives the values of jaccard of the n+1 largest values
    jaccard_values = heapq.nlargest(length+1, distance_recipes[i])
    jaccard_values.pop(0)        
    scores_1=[]
    
    ingredients_without_current = list(set(unique_ingredients_without_hidden) - set(recipes[i]))
    
    for j,obj in enumerate(unique_ingredients_without_hidden):
        if unique_ingredients_without_hidden[j] in recipes[i]:
            scores_1.append(0)
        else:
            index_of_ingredient = unique_ingredients_without_hidden.index(unique_ingredients_without_hidden[j])

            jaccard_dot_matrix_sum = 0
            jaccard_sum = 0

            for k in range(length):
                m = most_similar[k]
                jaccard_dot_matrix_sum += jaccard_values[k] * vectorized_matrix_m[m][index_of_ingredient]
                jaccard_sum += jaccard_values[k]

            if jaccard_sum != 0:    
                scores_1.append(jaccard_dot_matrix_sum/jaccard_sum)
            else:
                scores_1.append(0) 
    scores_ucf.append(scores_1)


In [ ]:
found_sum = 0
for j,obj in enumerate(recipes):
    
    hidden_element = list(set(recipes_before_removing_element[j]) - set(recipes[j]))
    #for K=1 we just want the maximum value
    max_value = max(scores_ucf[j])
    max_index = scores_ucf[j].index(max_value)

    ingredient_proposed_by_the_algorithm = unique_ingredients_without_hidden[max_index]

    if ingredient_proposed_by_the_algorithm == hidden_element[0]:
        found_sum += 1

print("precision:",found_sum/len(recipes))

In [ ]:
#jaccard of each ingredient with the other
distance_ingredients=[]
for i in unique_ingredients:
    jaccard_of_ingredients = []
    for j in unique_ingredients:
        set1, set2 = set(i), set(j)
        if(len(set1 | set2)==0):
            jaccard_of_ingredients.append(0)
        else:
            jaccard_of_ingredients.append(len(set1 & set2) / len(set1 | set2))
    distance_ingredients.append(jaccard_of_ingredients)